# 1. QSL4A简介

QSL4A（QPython Scripting Layer for Android）是 QPython 团队在 SL4A 项目基础上开发的扩展库，旨在为 Android 开发者提供更灵活的脚本编程环境。通过 QSL4A，开发者不仅可以使用 Python 快速实现 Android 应用的原型，还可以通过调用 Android API，实现设备控制和自动化任务。

## Hello World示例

In [ ]:
from androidhelper import Android
# 调用 Android API
droid = Android()
# 弹窗提示
droid.makeToast("Hello World!")

# 2. 用户界面交互

## 弹出消息提示

makeToast 是用于显示 Android Toast 消息的一个方法。Toast 是一种短暂的消息提示，不会阻塞用户操作，常用于提供简单的反馈或通知。

默认情况下，SL4A 的 makeToast 方法只支持短暂（short）显示，无法直接控制 Toast 显示的时间长度。

In [ ]:
droid.makeToast("Hello, QPython!")

## 剪贴板

QSL4A 提供了一个简便的接口来操作剪贴板，通常用于与剪贴板交互，复制和粘贴文本数据。 

In [ ]:
# 获取剪贴板结果
clipboard = droid.getClipboard().result

In [ ]:
# 设置剪贴板
droid.setClipboard("Hello QPython")

## 对话框

## 常见对话框API

### `dialogCreateAlert`- 创建简单的警告对话框

参数:
- `title`: 对话框标题。
- `message`: 对话框内容

In [ ]:
droid.dialogCreateAlert("提示", "是否继续？")
droid.dialogSetPositiveButtonText("是")
droid.dialogSetNegativeButtonText("否")
droid.dialogShow()  # dialogShow() 用于显示创建的对话框。
response = droid.dialogGetResponse().result  # dialogGetResponse() 返回用户的交互结果
print(response)  # 根据用户选择返回结果

### `dialogCreateInput`-允许用户输入的对话框

参数:
- `title`: 对话框标题。
- `message`: 提示内容。
- `defaultText`: 默认显示文本（可选）。
- `password`: 是否为密码输入模式（可选，默认False）

In [ ]:
# 创建基本的输入对话框
# 参数依次为：标题、提示信息、默认输入值
droid.dialogCreateInput("输入", "请输入姓名:", "默认名字")

# 设置按钮
droid.dialogSetPositiveButtonText("确定")
droid.dialogSetNegativeButtonText("取消")

# 显示对话框
droid.dialogShow()

# 获取用户输入的结果
response = droid.dialogGetResponse().result

# 判断用户操作并获取输入值
if response.get('which') == 'positive':
    # 获取用户输入的文本
    input_value = droid.dialogGetInput().result
    print("用户输入:", input_value)
elif response.get('which') == 'negative':
    print("用户取消输入")

### `dialogSetItems` - 带有选项列表的对话框

参数:
- `title`: 对话框标题。
- `items`: 一个列表，表示可选项。

In [ ]:
options = ["选项1", "选项2", "选项3"]
droid.dialogCreateAlert("选择一个选项")
droid.dialogSetItems(options)
droid.dialogShow()
response = droid.dialogGetResponse().result
print("选择的索引是:", response["item"])

### `dialogCreateDatePicker`- 日期选择器对话框

In [ ]:
droid.dialogCreateDatePicker(2024, 11, 26)  # 年，月，日
droid.dialogShow()
response = droid.dialogGetResponse().result
print("选择的日期是:", response["year"], response["month"], response["day"])

### `dialogCreateTimePicker`- 时间选择器对话框

In [ ]:
droid.dialogCreateTimePicker(12, 30)  # 小时, 分钟
droid.dialogShow()
response = droid.dialogGetResponse().result
print("选择的时间是:", response["hour"], response["minute"])

## 对话框的操作

### `dialogShow()`- 显示创建的对话框
`dialogGetResponse()`- 返回用户的交互结果

 `response.result` 可能包含的键： 
- `which`: 表示点击的按钮（'positive'/'negative'/'neutral'）
- `item`: 列表选择时的索引
- `canceled`: 对话框是否被取消
### `dialogGetInput()`- 获取用户输入内容
通常与其他对话框方法配合使用，能够实现各种用户输入场景。
- 返回值是一个 `Result` 对象
- 使用 `.result` 获取实际的输入文本
- 必须在 `dialogShow()` 之后使用
- 必须在获取按钮响应后使用

### `dialogGetPassword()`- 获取输入的密码

In [ ]:
droid.dialogCreateInput("设置密码", "请输入密码:", "", 'True')
droid.dialogSetPositiveButtonText("确定")
droid.dialogShow()

response = droid.dialogGetResponse().result
if response.get('which') == 'positive':
    password1 = droid.dialogGetPassword().result
    print("用户输入了: ", password1)
else:
    print("用户取消输入")

## `Vibrate` - 控制设备振动

In [ ]:
# 基本用法 - 振动指定时长(毫秒)
droid.vibrate(300)  # 振动300毫秒

# 3. 传感器和硬件访问

QSL4A 提供了一套传感器API，允许开发者访问设备的各种传感器数据，如加速度计、陀螺仪、磁力计、光传感器等。这些API可以用于实现与环境交互的应用，如游戏、导航或运动检测。 

## 传感器API的主要功能
1. 启用传感器监听
  - 开始监听指定传感器的数据。
2. 获取传感器数据
  - 通过回调函数或轮询方式获取实时数据。
3. 禁用传感器监听
  - 停止监听指定传感器的数据。

## 常用传感器API

### `startSensingTimed(sensorNumber, delayTime)` - 启动传感器监听，并指定数据采样的时间间隔

参数:
- `sensorNumber`: 要采集的传感器类型，1 = 所有传感器, 2 = 加速度计, 3 = 磁力仪, 4 = 光感 5 = 计步器, 6 = 陀螺仪
- `delayTime`：采样间隔，单位为毫秒（如 1000 表示每秒采样一次）。

### `readSensors()`- 获取最新的传感器数据

### `stopSensing()`- 停止传感器监听

In [ ]:
droid.startSensingTimed(4, 1000)  # 每秒采集一次数据
# 读取传感器结果
result = droid.readSensors().result
# 关闭传感器
droid.stopSensing()
print(result)

## 电池信息

### `batteryGetStatus()`- 获取电池状态

In [ ]:
battery_status = droid.batteryGetStatus().result
print(battery_status)

### `batteryGetLevel()`- 获取电池电量

In [ ]:
battery_level = droid.batteryGetLevel().result
print(battery_level)

### `batteryGetHealth()`- 获取电池健康状态

In [ ]:
battery_health = droid.batteryGetHealth().result
print(battery_health)

### `batteryGetTemperature()`- 获取电池温度

In [ ]:
battery_temp = droid.batteryGetTemperature().result
print(battery_temp)

### `batteryGetPlugType()`- 获取充电类型

In [ ]:
charging_type = droid.batteryGetPlugType().result
print(charging_type)

## 位置服务

### `startLocating(minUpdateTime, minUpdateDistance)`- 启动GPS定位
参数：
- `minUpdateTime`： 最小更新时间（毫秒）
- `minUpdateDistance`： 最小更新举例（米）
### `readLocation()`- 获取位置信息
### `stopLocation()`- 停止定位
### `getLastKnownLocation()`- 返回设备的最后已知位置

In [ ]:
# 启动位置服务
droid.startLocating(1000, 5)
# 获取位置信息
location = droid.readLocation().result
# 使用完后停止定位
droid.stopLocating()
print(location)

## 系统硬件信息

### `getDeviceId()`- 获取设备ID

In [ ]:
device_id = droid.getDeviceId().result
print(device_id)

### `getSimState()` - 获取SIM卡信息

In [ ]:
sim_info = droid.getSimState().result
print(sim_info)

### `getNetworkOperator()` - 返回当前注册运营商的数字名称 (MCC+MNC)

In [ ]:
net_operator = droid.getNetworkOperator().result
print(net_operator)

### `getNetworkOperatorName()` - 返回当前注册运营商的名称

In [ ]:
net_operator_name = droid.getNetworkOperatorName().result
print(net_operator_name)

# 4. 系统服务接口

## makeIntent - 创建 Intent 活动消息对象

在QSL4A中，makeIntent 方法用于创建一个 Android Intent 对象。Intent 是 Android 中用于描述要执行的操作（如启动一个活动或服务）的消息对象。
### 主要用途：
1. 启动活动（Activity）
使用 Intent 启动指定的 Android 活动，例如打开特定的应用或系统设置。
2. 传递数据
在 Intent 中携带数据，通过键值对的方式传递给目标组件。
3. 触发特定动作
创建一个 Intent 并指定操作（如 ACTION_VIEW 或 ACTION_SEND），触发与操作相关的系统或应用行为。
### 参数说明：
- `action`：Intent 的动作类型，通常使用 Android 定义的标准动作，如 android.intent.action.VIEW。
- `uri`：目标资源的 URI，指定资源路径或文件路径。
- `type`：可选，默认为None。指定数据类型（如 image/png）。
- `extras`：可选，默认为None。额外的数据，作为键值对传递。
- `categories`: 可选，默认为None。类别列表
- `packagename`: 可选，默认为None。包名
- `classname`: 可选，默认为None。类名
- `flags`: 可选，默认为None。Intent 标志

In [ ]:
# 创建基本Intent：
intent = droid.makeIntent(
    "android.intent.action.VIEW",
    "https://www.example.com",
    "text/html",
    {"extra_key": "value"}
)

In [ ]:
# 创建应用内跳转Intent:
intent = droid.makeIntent(
    "android.intent.action.MAIN",
    None,
    None,
    None,
    ["android.intent.category.LAUNCHER"],
    "com.example.app",
    "com.example.app.MainActivity"
)

## `startActivityIntent()` - 启动Android活动

通过 makeIntent动态生成 Intent 后，就可以传递给 startActivityIntent()，以便启动活动完成指定操作。

In [ ]:
# 打开一个 URL
action = "android.intent.action.VIEW"
uri = "https://www.baidu.com"
intent = droid.makeIntent(action, uri)
droid.startActivityIntent(intent.result)

In [ ]:
# 打开手机拨号应用并填充号码  
action = "android.intent.action.DIAL"
uri = "tel:123456789"
intent = droid.makeIntent(action, uri)
droid.startActivityIntent(intent.result)

In [ ]:
# 向其他应用发送文本
action = "android.intent.action.SEND"
extras = {"android.intent.extra.TEXT": "Hello from QPython!"}
intent = droid.makeIntent(action, None, None, extras)
droid.startActivityIntent(intent.result)

## `sendBroadcastIntent()` - 发送广播消息

In [ ]:
# 创建可重用的广播Intent
import time

def create_notification_intent(message, priority):
    return droid.makeIntent(
        "com.example.NOTIFICATION",
        None,
        None,
        {
            "message": message,
            "priority": priority,
            "timestamp": time.time()
        }
    )

# 使用
notification = create_notification_intent("新消息", "high")
if notification.result:
    droid.sendBroadcastIntent(notification.result)

In [ ]:
# 批量广播发送
def create_broadcast_intents(messages):
    intents = []
    for msg in messages:
        intent = droid.makeIntent(
            "com.example.MESSAGE",
            None,
            "text/plain",
            {"content": msg}
        )
        if intent.result:
            intents.append(intent.result)
    return intents

# 使用
messages = ["msg1", "msg2", "msg3"]
intents = create_broadcast_intents(messages)
for intent in intents:
    droid.sendBroadcastIntent(intent)

## Preferences - 偏好设置

### `prefGetValue(key, filename)` - 从共享偏好设置中读取值

参数：
- `key`: 首选项的键名称
- `filename`: （可选）所需的首选项文件。如果未定义，则使用默认的共享首选项。

In [ ]:
pref_username = droid.prefGetValue("username")
print(pref_username)

### `prefPutValue(key, value, filename)` - 将值写入共享首选项

参数：
- `key`: 首选项的键名称
- `value`: 要存储的值（可以是字符串、整数、布尔值等）
- `filename`: （可选）所需的首选项文件。如果未定义，则使用默认的共享首选项。

In [ ]:
droid.prefPutValue("username", "QPython")

### `prefGetAll(filename)` - 获取共享偏好值列表

参数：
- `filename`: （可选）所需的首选项文件。如果未定义，则使用默认的共享首选项。

In [ ]:
pref_all = droid.prefGetAll()
print(pref_all)

## ApplicationManager - 应用管理

### `getLaunchablePackages()` - 返回所有可启动应用程序类名的列表

In [ ]:
apps = droid.getLaunchablePackages()
print(apps)

### `launch(classname, packagename, wait)` - 启动指定应用

参数：
- `classname`: 应用程序类名
- `packagename`: 应用程序包名
- `wait`: 是否等待应用启动完毕，默认是True

In [ ]:
droid.launch(packagename='com.huawei.himovie')

### `getRunningPackages()` - 获取正在运行的应用列表

In [ ]:
apps = droid.getRunningPackages()
print(apps)

### `forceStopPackage(packageName)` - 强行停止运行应用

参数：
- `packageName`: 应用程序包名

In [ ]:
droid.forceStopPackage(  packageName='com.huawei.himovie')

# 5. 通信功能

## SMS - 短信

### `smsSend(destinationAddress, text)` - 发送短信

参数：
- `destinationAddress`：目标电话号码
- `text`：短信文本内容

In [ ]:
droid.smsSend('19113949755', '你好')

### `smsGetMessageCount(unreadOnly, folder)` - 获取短信数量

参数：
- `unreadOnly`: bool型，是否筛选未读状态的短信
- `folder`: 短信文件夹，默认为"inbox"

In [ ]:
msg_count = droid.smsGetMessageCount(False)

### `smsGetMessageIds(unreadOnly, folder)` - 获取所有短信id

参数：
- `unreadOnly`: bool型，是否筛选未读状态的短信
- `folder`: 短信文件夹，默认为"inbox"

In [ ]:
msg_ids = droid.smsGetMessageIds(False)

### `smsGetMessages(unreadOnly, folder, attributes)` - 获取所有短信

参数：
- `unreadOnly`: bool型，是否筛选未读状态的短信
- `folder`: 短信文件夹，默认为"inbox"
- `attributes`: 参数列表，选填

In [ ]:
msg = droid.smsGetMessages(False)

### `smsGetMessageById(id, attributes)` - 根据id获取短信参数

参数：
- `id`: 短信id
- `attributes`: 参数列表，选填

In [ ]:
msg_ids = droid.smsGetMessageById(1)

### `smsGetAttributes()` - 获取所有可能的短信参数

In [ ]:
attrs = droid.smsGetAttributes()

### `smsDeleteMessage(id)` - 根据id删除短信

In [ ]:
droid.smsDeleteMessage(1)

### `smsMarkMessageRead(ids, read)` - 将短信标为已读/未读

参数：
- `id`: 短信id
- `read`: bool型，True-已读，False-未读

In [ ]:
droid.smsMarkMessageRead([2, 3], True)

## Phone - 拨打电话

### `phoneCall(uri)` -  通过URI呼叫联系人/电话号码
某些 Android 系统会在拨打电话之前弹出确认界面，以防止未经用户同意拨打电话。
参数：
- `uri`（字符串）：电话号码或包含 tel: 前缀的 URI。  例如：
  - 1234567890（本地号码）
  - tel:1234567890（URI 格式，推荐）
  - tel:+861234567890（国际号码）

In [ ]:
droid.phoneCall(uri)

### `phoneCallNumber(number)` - 发起电话呼叫
会跳过拨号界面，直接拨打指定号码，而无需用户进一步确认。
参数：
- `number`（字符串）：要拨打的电话号码。需要包含国家代码（如果是国际号码）或者本地号码。

In [ ]:
droid.phoneCallNumber('12345657890')

### `phoneDial(uri)` - 打开拨号界面，并输入指定URI
用于打开拨号界面并填入指定号码的方法，但不会自动拨出电话。这提供了一种较为安全的方式，让用户自行决定是否拨打电话。

适用场景：
- 在需要提示用户拨打电话的情况下使用，而不是直接自动拨号。
- 应用程序希望更安全地提供拨号功能，避免未经用户确认直接拨号。

In [ ]:
# 设置要拨打的号码
phone_number = "tel:1234567890"

# 打开拨号界面
droid.phoneDial(phone_number)

### `phoneDialNumber(number)` - 打开拨号界面并填入指定的电话号码
一种更简洁的方法，适合快速实现拨号界面预填功能，同时确保用户有机会确认操作，从而提升用户体验和安全性。
  
参数：
- `number`（字符串）：需要填入拨号界面的电话号码。可以是本地号码或国际号码（带国家代码）。

In [ ]:
# 要拨打的电话号码
phone_number = "1234567890"

# 打开拨号界面并填入电话号码
droid.phoneDialNumber(phone_number)

## WiFi

### `wifiGetScanResults()` - 获取wifi扫描结果

- 参数：无。
- 返回值：包含扫描结果的列表，每个网络是一个字典，包含以下信息：
  - BSSID：接入点的 MAC 地址。
  - SSID：网络名称。
  - capabilities：加密方式。
  - frequency：频率。
  - level：信号强度（dBm）。

In [ ]:
res= droid.wifiGetScanResults().result
print(res)

### `wifiStartScan()` - 启动wifi网络扫描

In [ ]:
droid.wifiStartScan()

### `checkWifiState()` - 检查Wifi状态

In [ ]:
status = droid.checkWifiState().result
print(status)

### `toggleWifiState(enabled)` - 返回有关当前活动接入点的信息

In [ ]:
res= droid.toggleWifiState(True).result
print(res)

### `wifiDisconnect()` - 断开当前的 WiFi 连接

In [ ]:
droid.wifiDisconnect()

### `wifiGetConnectionInfo()` - 获取当前的 WiFi 连接信息

- 参数：无。
- 返回值：包含当前连接信息的字典，常见字段包括：
  - `ssid`：当前连接的网络名称。
  - `bssid`：接入点的 MAC 地址。
  - `ip_address`：设备的 IP 地址。
  - `link_speed`：连接速度（Mbps）。

In [ ]:
# 获取当前 WiFi 连接信息
info = droid.wifiGetConnectionInfo().result
print(info)

### `wifiReassociate()` - 返回有关当前活动接入点的信息

In [ ]:
res= droid.wifiReassociate().result
print(res)

### `wifiReconnect()` - 重新连接到当前已配置的 WiFi 网络

In [ ]:
droid.wifiReconnect()

# 6. 多媒体处理

## Camera - 照相机操作

### `cameraCapturePicture(targetPath)` - 拍照并保存
参数：
- targetPath: 照片保存路径，选填。

返回值：
- 如果成功，返回照片文件的路径。
- 如果失败，返回 None。

In [ ]:
res = droid.cameraCapturePicture()
print(res)

### `cameraInteractiveCapturePicture(targetPath)` - 启动交互式相机界面
参数：
- `targetPath`：字符串，照片保存的完整路径。

- 返回值：
- 如果成功，返回照片文件的路径。
- 如果失败，返回 None。

In [ ]:
# 设置保存路径
photo_path = "/sdcard/interactive_photo.jpg"

# 调用交互式相机拍照
result = droid.cameraInteractiveCapturePicture(photo_path)

if result.result:
    print(f"照片已保存至：{photo_path}")
else:
    print("拍照失败")

## MediaPlay - 音频播放

### `mediaPlay(url)` - 播放音频文件
参数：
- url：字符串，音频文件的路径（支持本地文件或 URL）。

返回值：无显式返回值。

支持格式：常见音频格式，如 MP3、WAV。

In [ ]:
# 播放音频文件
audio_path = "/sdcard/sample.mp3"
droid.mediaPlay(audio_path)

### `mediaPlayClose()` - 停止播放当前的音频文件

In [ ]:
droid.mediaPlayClose()

### `mediaPlayPause()` - 暂停当前播放的音频文件

In [ ]:
droid.mediaPlayPause()

### `mediaPlaySeek(msec)` - 调整播放进度到指定位置
参数：
- `msec`：整数，音频的播放位置（以毫秒为单位）

In [ ]:
# 设置播放位置为 10 秒（10000 毫秒）
droid.mediaPlaySeek(10000)

## Ringer & Media Volume - 铃声 & 媒体音量

Ringer Volume 和 Media Volume 是两个不同的音量设置，它们控制设备的两种主要音量：
- Ringer Volume（铃声音量）：控制设备的铃声、通知音量。
- Media Volume（媒体音量）：控制设备播放音频或视频时的音量，如音乐播放器、视频播放器、游戏等。

### `getMaxRingerVolume()` - 获取铃声音量最大值
- 参数：无。
- 返回值：整数，表示当前铃声音量级别

In [ ]:
res = droid.getMaxRingerVolume().result
print(res)

### `getRingerVolume()` - 获取当前的铃声音量
- 参数：无。
- 返回值：整数，表示当前铃声音量级别

In [ ]:
res = droid.getRingerVolume().result
print(res)

### `setRingerVolume(volume)` - 设置铃声音量  
参数：
- `volume`：整数，目标铃声音量级别

In [ ]:
droid.setRingerVolume(2)

### `getMaxMediaVolume()` - 获取最大媒体音量
- 参数：无。
- 返回值：整数，表示当前媒体音量级别（通常是 0 到 15 之间的整数，具体范围取决于设备）。

In [ ]:
res = droid.getMaxMediaVolume().result

### `getMediaVolume()` - 获取当前媒体音量
- 参数：无。
- 返回值：整数，表示当前媒体音量级别（通常是 0 到 15 之间的整数，具体范围取决于设备）。

In [ ]:
res = droid.getMediaVolume().result
print(res)

### `setMediaVolume(volume)` - 设置媒体音量
- 参数：
  - `volume`：整数，目标媒体音量级别（通常是 0 到 15 之间的整数，具体范围取决于设备）。
- 返回值：无显式返回值。

In [ ]:
droid.setMediaVolume(2)

### `toggleRingerSilentMode()` - 切换静音模式

In [ ]:
droid.toggleRingerSilentMode()

## Speech - 语音

### `ttsSpeak(message)` - 文本转语音
- 参数：
  - `message`：字符串，要转换为语音的文本。
- 返回值：无显式返回值。

In [ ]:
droid.ttsSpeak("Hello, this is a text-to-speech example.")

### `ttsIsSpeaking()` - 检测是否正在处理语音

In [ ]:
res = droid.ttsIsSpeaking().result
print(res)

### `speechToText(RecordSecond=10, AmrFile=None, Language=None)` - 语音转文本
- 参数：
  - `RecordSecond`: 录音时长
  - `AmrFile`: 临时存储的amr文件
  - `Language`: 选择的语言，为空时使用系统默认语言
当 `RecordSecond=None` 时，使用 `AmrFile` 的值。

In [ ]:
res = droid.speechToText()
print(res)

### `recorderStartMicrophone(targetPath)` - 开始录音
- 参数
  - `targetPath`: 录音文件保存地址，选填，为None时使用默认地址
### `recorderStop()` - 结束录音

In [ ]:
import time

droid.ttsSpeak("Start recording")
# 开始录音
result = droid.recorderStartMicrophone()
# 等待录音5秒
time.sleep(5)
droid.ttsSpeak("End of recording")
# 结束录音
droid.recorderStop()
# 获取录音结果
recording_result = result.result

# 7. FullScreenWrapper2 基础

FullScreenWrapper2 是用于 Android 开发的一个控件封装工具，通常用来帮助开发者以全屏模式显示界面元素。在 Android 开发中，基于 QSL4A，FullScreenWrapper2 可以被用来创建全屏显示的界面，适用于需要全屏显示的应用场景。 
 
在 FullScreenWrapper2 中，你可以通过结合 XML 布局文件和 Python 代码来实现更复杂的界面设计。FullScreenWrapper2 本身是用于将控件显示在全屏模式下，但你可以通过 XML 定义布局并在 Python 中加载和管理这些布局。

## 导入模块

In [ ]:
import androidhelper
from qsl4ahelper.fullscreenwrapper2 import *

## 定义 XML 布局

In [ ]:
content = """<?xml version="1.0" encoding="utf-8"?>
<LinearLayout
	android:layout_width="fill_parent"
	android:layout_height="fill_parent"
	android:background="#7f3faf"
	android:orientation="vertical"
	xmlns:android="http://schemas.android.com/apk/res/android">
	<ScrollView   
        android:layout_width="fill_parent"   
        android:layout_height="50dp"
        android:layout_weight="1" > 
    <TextView
	android:id="@+id/Title"
	android:layout_width="fill_parent"
	android:layout_height="wrap_content"
	android:textSize="7dp"
	android:text="Hello QPython"
	android:background="#af7f3f"
	android:textColor="#ffffff"
	android:textStyle="bold"
	android:gravity="center"
	/>
    </ScrollView>
    <ListView
        android:id="@+id/listview"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_weight="20"
        android:background="#7f3faf"
    />
    <Button
	android:layout_width="fill_parent"
	android:layout_height="50dp"
	android:text="Exit"
	android:id="@+id/but_cancle"
	android:textAllCaps="false"
	android:textSize="5dp"
	android:background="#3faf7f"
	android:textColor="#ffffff"
	android:layout_weight="1"
	android:gravity="center"/>
</LinearLayout>
"""

在这个 XML 布局中，我们定义了一个包含 LinearLayout、ScrollView、TextView、ListView 和 Button 的界面。 
- LinearLayout：根布局，垂直方向排列子元素。
- ScrollView：包含 TextView，用于显示可滚动的文本。
- TextView：显示欢迎消息 "Hello QPython"。
- ListView：列表视图，用于显示项目。
- Button：一个退出按钮，点击后将触发事件。

## 定义 Layout

`Layout` 是一个基础类，用于创建和管理应用界面的布局。它是 FullScreenWrapper2 框架的一部分，用于将 UI 组件（如按钮、文本框、列表视图等）放置到屏幕上，并控制它们的显示和交互。

In [ ]:
class MainScreen(Layout):
    def __init__(self):
        super(MainScreen, self).__init__(content, "IndexApp")

    def on_show(self):
        self.views.but_cancle.add_event(click_EventHandler(self.views.but_cancle, self.exit))

    def on_close(self):
        pass

    def exit(self, view, dummy):
        droid = FullScreenWrapper2App.get_android_instance()
        droid.makeToast("Exit")
        FullScreenWrapper2App.close_layout()

`MainScreen` 类继承自 `Layout`，表示应用的主界面。
- `__init__` 方法：通过 `super()` 初始化 `Layout` 类，并传入布局文件 `content` 和应用名称 "IndexApp"。
- `on_show` 方法：当界面显示时，会给 `but_cancle` 按钮添加点击事件，点击时会调用 `exit` 方法。
- `on_close` 方法：该方法目前为空，表示在关闭界面时没有特定操作。
- `exit` 方法：点击 "Exit" 按钮时，调用此方法。它会触发以下操作：
- 使用 `FullScreenWrapper2App.get_android_instance()` 获取 Android 实例。
- 使用 `droid.makeToast("Exit")` 显示一个短暂的提示消息 "Exit"。
- 使用 `FullScreenWrapper2App.close_layout()` 关闭当前布局界面。

## 主程序

In [ ]:
if __name__ == '__main__':
    # 创建一个 Android 实例，用于与 Android 系统交互
    droid = androidhelper.Android()
    # 初始化 FullScreenWrapper2App，并传入 Android 实例。
    FullScreenWrapper2App.initialize(droid)
    # 显示 MainScreen 布局。
    FullScreenWrapper2App.show_layout(MainScreen())
    # 启动事件循环，等待用户交互。
    FullScreenWrapper2App.eventloop()

# 8. FullScreenWrapper2 布局

## Layout的作用

### 1. 定义界面结构
Layout 类负责通过传入的 XML 布局定义界面结构。例如，XML 字符串定义了一个包含 LinearLayout、ScrollView、TextView、ListView 和 Button 的布局结构。Layout 类将这些 UI 组件根据 XML 的配置进行实例化，并将它们放置到屏幕上。  
### 2. 管理UI控件
Layout 会管理所有的 UI 组件（如 Button、TextView 等），并提供访问它们的方式。例如，在 `MainScreen` 类中的 `on_show` 方法里，通过 `self.views.but_cancle` 访问了 Button 组件，并为它绑定了点击事件。  
### 3. 事件处理
Layout 类还提供了事件处理机制，可以将用户的交互（如点击按钮、滑动列表等）与对应的处理函数绑定起来。在 `MainScreen` 类中，s`elf.views.but_cancle.add_event(click_EventHandler(self.views.but_cancle, self.exit))` 就是为 "Exit" 按钮添加了点击事件处理程序，点击按钮时会执行 exit 方法。  
### 4. 界面显示与关闭
Layout 还负责界面的显示与关闭。在 `FullScreenWrapper2App.show_layout(MainScreen())` 调用中，Layout 将会加载并显示 `MainScreen` 布局。
在 `exit` 方法中，调用了 `FullScreenWrapper2App.close_layout()` 来关闭当前的界面布局。

## Layout的基本用法

### 1. 构造函数 __init__
Layout 类的构造函数允许你定义应用界面的布局和其他属性。你可以传递一个 XML 字符串来设置布局，也可以传递其他参数。
```python
class MainScreen(Layout):
    def __init__(self):
        # 通过传入 XML 布局定义来初始化 Layout
        super(MainScreen, self).__init__(content, "IndexApp")
```
- `content`：传入一个 XML 格式的布局字符串，定义界面的结构。
- "IndexApp"：这是应用的名称，用于界面的标题或其他标识。
### 2. XML 布局
Layout 使用 XML 字符串来定义界面布局。这些 XML 字符串定义了界面的结构，例如：
- LinearLayout：根布局，用来管理其他 UI 元素的排列。
- TextView：显示文本。
- Button：按钮元素。
```xml
<LinearLayout
  android:layout_width="fill_parent"
  android:layout_height="fill_parent"
  android:background="#7f3faf"
  android:orientation="vertical"
  xmlns:android="http://schemas.android.com/apk/res/android">
  <Button
    android:id="@+id/but_cancle"
    android:layout_width="fill_parent"
    android:layout_height="50dp"
    android:text="Exit"
    android:background="#3faf7f"
    android:textColor="#ffffff"
    android:gravity="center"/>
</LinearLayout>
```
### 3. 访问视图组件
Layout 类会根据提供的 XML 布局初始化所有视图组件。在界面加载之后，你可以通过 self.views 属性来访问这些组件。例如：
```python
self.views.but_cancle
```
这段代码会访问布局中 ID 为 `but_cancle` 的按钮视图，允许你对它进行操作，比如添加事件监听。  
### 4. 事件处理
Layout 类允许你为视图组件添加事件处理程序，例如点击事件、长按事件等。通过 add_event 方法，你可以将视图组件与特定的事件处理函数绑定起来。
```python
def on_show(self):
    # 为按钮添加点击事件处理程序
    self.views.but_cancle.add_event(click_EventHandler(self.views.but_cancle, self.exit))

def exit(self, view, dummy):
    # 按钮点击时执行的操作
    droid = FullScreenWrapper2App.get_android_instance()
    droid.makeToast("Exit")
    FullScreenWrapper2App.close_layout()
```
### 5. 显示和关闭布局
Layout 类还提供了方法来显示和关闭界面：
- `FullScreenWrapper2App.show_layout(MainScreen())`：显示由 `MainScreen` 类定义的布局。
- `FullScreenWrapper2App.close_layout()`：关闭当前显示的布局界面。

## Layout 类的常用方法

- `add_event`：
  - 用于将事件处理函数与视图组件绑定。
- `on_show`：
  - 当布局显示时会调用此方法。你可以在这个方法中添加视图组件的事件监听器或执行初始化任务。
- `on_close`：
  - 当布局关闭时会调用此方法。你可以在这里执行资源清理或其他关闭时的操作。
- `views`：
  - `views` 是一个包含所有视图组件的字典。你可以通过 `self.views` 来访问布局中的各个组件。

# 9. FullScreenWrapper2 事件

## EventHandler 类

`EventHandler` 是一个通用的事件处理器，它可以用于捕获和匹配 SL4A 事件。事件数据通常是一个字典，包含事件名称 (name)、事件数据 (data) 和时间戳 (time)。事件处理器会根据事件名称和数据中的属性值来决定是否触发对应的处理函数。  
### 构造函数：
```python
def __init__(self, event_name, compare_attribute, compare_value, view=None, handler_function=None):
```
- `event_name`: 事件的名称，如 "click"、"itemclick"、"key" 等。
- `compare_attribute`: 要在事件数据中匹配的属性名，通常是 "id"、"key" 等。如果为 None，则只匹配事件名称。
- `compare_value`: 与 `compare_attribute` 对应的值，用于进一步匹配事件。
- `view`: 相关视图的引用，事件处理程序会触发时会传入该视图。
- `handler_function`: 事件匹配时要调用的处理函数。
### 方法：
- `match_event_data(self, event_data)`:
  - 用于匹配传入的 `event_data` 是否符合事件处理器的条件。事件数据是一个字典，包含 "name" 和 "data" 两个字段。该方法首先匹配事件名称，然后检查 "data" 中的属性值是否与 `compare_value` 匹配。
- `__str__(self)`:
  - 提供了一个字符串表示形式，便于调试和查看事件处理器的配置。

### 示例：
```python
event_handler = EventHandler("click", "id", "button1")
if event_handler.match_event_data(event_data):
    event_handler.handler_function(view, event_data)
```
 这里的 `event_handler` 会匹配事件名称为 "click"，且事件数据中的 "id" 为 "button1" 的事件。  

## click_EventHandler 类

`click_EventHandler` 是一个预定义的事件处理器，专门用于处理点击事件。它通常与视图（如 TextView、Button、ImageView 等）配合使用。当视图被点击时，会触发相应的事件处理函数。  
### 构造函数：
```python
def __init__(self, view, handler_function=None):
```
- `view`: 需要关联点击事件的视图组件。
- `handler_function`: 事件匹配时调用的处理函数。

`click_EventHandler` 类会自动根据视图的 `view_id` 和事件名称 "click" 来初始化 `EventHandler`。

### 示例：
```python
click_handler = click_EventHandler(button, button_click_handler)
```
当按钮被点击时，`button_click_handler` 函数会被调用。

## itemclick_EventHandler 类

`itemclick_EventHandler` 用于处理 `ListView` 的项点击事件。当用户点击 `ListView` 中的一项时，会触发相应的事件处理函数。
### 构造函数：
```python
def __init__(self, view, handler_function=None):
```
- `view`: 需要关联项点击事件的 ListView。
- `handler_function`: 事件匹配时调用的处理函数。

`itemclick_EventHandler` 类会自动根据 `ListView` 的 `view_id` 和事件名称 "itemclick" 来初始化 `EventHandler`。  
### 示例：
```python
item_click_handler = itemclick_EventHandler(listview, item_click_handler_function)
```
 当 `ListView` 中的某一项被点击时，`item_click_handler_function` 会被触发。

## key_EventHandler 类

`key_EventHandler` 用于处理按键事件。默认情况下，它会监听 "Back" 按键 (key_id = "4") 的事件，但你也可以指定其他按键。  
### 构造函数：
```python
def __init__(self, key_match_id="4", view=None, handler_function=None):45
```
- `key_match_id`: 要匹配的按键 ID，默认值为 "4"（即返回键）。
- `view`: 关联视图的引用，事件触发时会传入。
- `handler_function`: 事件匹配时调用的处理函数。

`key_EventHandler` 类会自动初始化 `EventHandler`，将 `event_name` 设置为 "key"，将 compare_attribute 设置为 "key"，并将 compare_value 设置为 `key_match_id`。
### 示例：
```python
key_handler = key_EventHandler("4", some_view, back_key_handler)
```
当按下返回键时，`back_key_handler` 会被触发。

# 10. 多功能信息dashboard

接下来，我们将基于QSL4A和FullScreenWrapper2，在QPython上创建一个简单图形用户界面。程序分为两个主要界面：主菜单界面和命令详情界面，实现了多种功能，比如显示设备状态、复制代码片段以及执行简单的设备操作。

In [ ]:
import androidhelper
from qsl4ahelper.fullscreenwrapper2 import *

# 菜单主界面布局定义
content = """<?xml version="1.0" encoding="utf-8"?>
<LinearLayout
	android:layout_width="fill_parent"
	android:layout_height="fill_parent"
	android:background="#7f3faf"
	android:orientation="vertical"
	xmlns:android="http://schemas.android.com/apk/res/android">
	<ScrollView   
        android:layout_width="fill_parent"   
        android:layout_height="50dp"
        android:layout_weight="1" > 
    <!-- 标题 -->
    <TextView
	android:id="@+id/Title"
	android:layout_width="fill_parent"
	android:layout_height="wrap_content"
	android:textSize="7dp"
	android:text="Dashboard"
	android:background="#af7f3f"
	android:textColor="#ffffff"
	android:textStyle="bold"
	android:gravity="center"
	/>
    </ScrollView>
    <!-- 菜单显示 -->
    <ListView
        android:id="@+id/listview"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_weight="20"
        android:background="#7f3faf"
    />
    <!-- 退出按键 -->
    <Button
	android:layout_width="fill_parent"
	android:layout_height="50dp"
	android:text="Exit"
	android:id="@+id/but_cancle"
	android:textAllCaps="false"
	android:textSize="5dp"
	android:background="#3faf7f"
	android:textColor="#ffffff"
	android:layout_weight="1"
	android:gravity="center"/>
</LinearLayout>
"""

# 命令详情界面布局定义
detail_content = """<?xml version="1.0" encoding="utf-8"?>
<LinearLayout
	android:layout_width="fill_parent"
	android:layout_height="fill_parent"
	android:background="#7f3faf"
	android:orientation="vertical"
	xmlns:android="http://schemas.android.com/apk/res/android">
	<!-- 标题 -->
	<ScrollView   
        android:layout_width="fill_parent"   
        android:layout_height="50dp"
        android:layout_weight="1" > 
    <TextView
	android:id="@+id/Title"
	android:layout_width="fill_parent"
	android:layout_height="wrap_content"
	android:textSize="7dp"
	android:text="Dashboard"
	android:background="#af7f3f"
	android:textColor="#ffffff"
	android:textStyle="bold"
	android:gravity="center"
	/>
    </ScrollView>
    <!-- 命令执行结果显示框 -->
    <ListView
        android:id="@+id/listview"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_weight="20"
        android:background="#7f3faf"
    />
    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="vertical"
        android:layout_margin="8dp">
        
        <!-- 代码显示框 -->
        <ScrollView
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:background="#ffffff"
            android:padding="8dp"
            android:elevation="4dp">
            <TextView
                android:id="@+id/codeTextView"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:lineSpacingExtra="3dp"
                android:text=""
                android:textColor="#000000"
                android:fontFamily="monospace"
                android:gravity="start"/>
        </ScrollView>
    </LinearLayout>
	<LinearLayout
		android:layout_width="fill_parent"
		android:layout_height="50dp"
		android:orientation="horizontal"
		android:layout_weight="8">
    <!-- 执行按键 -->
	<Button
		android:layout_width="fill_parent"
		android:layout_height="50dp"
		android:text="Try"
		android:id="@+id/but_try"
		android:textAllCaps="false"
		android:background="#007f7f"
		android:textColor="#ffffffff"
		android:layout_weight="1"
		android:gravity="center"/>
    <!-- 复制按键 -->
	<Button
		android:layout_width="fill_parent"
		android:layout_height="50dp"
		android:text="Copy"
		android:id="@+id/but_copy"
		android:textAllCaps="false"
		android:background="#7f7f00"
		android:textColor="#ffffffff"
		android:layout_weight="1"
		android:gravity="center"/>
	</LinearLayout>
	<!-- 返回按键 -->
	<Button
	android:layout_width="fill_parent"
	android:layout_height="50dp"
	android:text="Back"
	android:id="@+id/but_cancle"
	android:textAllCaps="false"
	android:textSize="5dp"
	android:background="#3faf7f"
	android:textColor="#ffffff"
	android:layout_weight="1"
	android:gravity="center"/>
</LinearLayout>
"""

# 全局变量，用于记录菜单界面选中的命令
cmd_position = 0

# 菜单界面显示的命令列表
INDEX_PAGE_LIST = [
    'Location',
    'SIM State',
    'WiFi State',
    'Battery State',
    'Vibrate',
    'Microphone',
    'Camera',
    'Sensors',
    'Application Manager'
]


class GetCommandResult(object):
    """执行命令，获取对应的结果和文档"""
    def __init__(self, cmd, droid):
        self.cmd = cmd
        self.droid = droid

    def get_result(self):
        function_name = f'_get_{self.cmd}'
        if hasattr(self, function_name):
            # 判断是否包含指定命令的执行方法，已有的方法则返回执行结果
            return getattr(self, function_name)()
        return []

    def get_doc(self):
        # 判断是否包含指定命令的执行方法，已有的方法则对应代码文档
        function_name = f'_get_{self.cmd}'
        if hasattr(self, function_name):
            return getattr(self, function_name).__doc__
        return ''

    def _get_location(self):
        """droid.startLocating(1000, 5)\n""" \
            """location = droid.readLocation().result\n""" \
            """droid.stopLocating()"""
        # 启动位置服务
        self.droid.startLocating(1000, 5)
        # 获取位置信息
        location = self.droid.readLocation().result
        # 使用完后停止定位
        self.droid.stopLocating()
        if location:
            network_data = location['network']
            show_list_items = [
                f'{key}: {value}'
                for key, value in network_data.items()
            ]
            return show_list_items
        return []

    def _get_wifi_state(self):
        """wifi_info = droid.wifiGetConnectionInfo().result"""
        # 获取wifi状态数据
        wifi_info = self.droid.wifiGetConnectionInfo().result
        show_list_items = [
            f'{key}: {value}'
            for key, value in wifi_info.items()
        ]  # 将获取到的所有结果显示到页面上
        return show_list_items

    def _get_sim_state(self):
        """sim_state = droid.getSimState().result\n""" \
            """network_operator = droid.getNetworkOperator().result\n""" \
            """network_operator_name = droid.getNetworkOperatorName().result"""
        show_list_items = [
            f'State: {self.droid.getSimState().result}',  # 获取SIM卡信息
            f'NetworkOperator: {self.droid.getNetworkOperator().result}',  # 返回当前注册运营商的数字名称 (MCC+MNC)
            f'NetworkOperatorName: {self.droid.getNetworkOperatorName().result}',  # 返回当前注册运营商的名称
        ]
        return show_list_items

    def _get_battery_state(self):
        """battery_status = droid.batteryGetStatus().result\n""" \
            """battery_level = droid.batteryGetLevel().result\n""" \
            """battery_temperature = droid.batteryGetTemperature().result\n"""
        show_list_items = [
            f'Battery Status: {self.droid.batteryGetStatus().result}',  # 获取电池状态
            f'Battery Level: {self.droid.batteryGetLevel().result}',  # 获取电池电量
            f'Battery Temperature: {self.droid.batteryGetTemperature().result}',  # 获取电池温度
        ]
        return show_list_items

    def _get_vibrate(self):
        """droid.vibrate(300)"""
        self.droid.vibrate(300)  # 振动300毫秒
        return ['Vibrate(300)']

    def _get_microphone(self):
        """droid.ttsSpeak("Start recording")\n"""\
            """result = droid.recorderStartMicrophone()\n"""\
            """time.sleep(5)\n"""\
            """droid.ttsSpeak("End of recording")\n"""\
            """droid.recorderStop()\n"""\
            """recording_result = result.result\n"""\
            """droid.mediaPlay(recording_result)"""
        self.droid.ttsSpeak("Start recording")
        # 开始录音
        result = self.droid.recorderStartMicrophone()
        # 等待录音5秒
        time.sleep(5)
        self.droid.ttsSpeak("End of recording")
        # 结束录音
        self.droid.recorderStop()
        # 获取录音结果
        recording_result = result.result
        # 播放录音文件
        self.droid.mediaPlay(recording_result)
        return [f'Voice: {recording_result}']

    def _get_camera(self):
        """droid.cameraCapturePicture().result"""
        # 打开相机拍照
        result = self.droid.cameraCapturePicture().result
        return [f'Picture: {result}']

    def _get_sensors(self):
        """droid.startSensingTimed(4, 1000)"""
        # 开启传感器
        self.droid.startSensingTimed(4, 1000)
        # 读取传感器结果
        result = self.droid.readSensors().result
        # 关闭传感器
        self.droid.stopSensing()
        return [f'{key}: {value}'
                for key, value in result.items()]

    def _get_application_manager(self):
        """running_app = droid.getRunningPackages().result\n"""\
            """launchable_packages = droid.getLaunchablePackages().result"""
        # 获取正在运行的应用
        running_app = self.droid.getRunningPackages().result
        # 获取所有可启动应用程序类名的列表
        launchable_packages = self.droid.getLaunchablePackages().result
        launchable_packages = '\n' + '\n'.join([f'{key}: {val}' for key, val in launchable_packages.items()])
        return [f'Running Packages: {";".join(running_app)}',
                f'Launchable Packages: {launchable_packages}']


class MainScreen(Layout):
    def __init__(self):
        super(MainScreen, self).__init__(content, "IndexApp")

    def on_show(self):
        list_view = self.views.listview
        # 将菜单列表设置到主界面
        list_view.set_listitems(INDEX_PAGE_LIST)
        # 为菜单项添加点击事件
        list_view.add_event(itemclick_EventHandler(list_view, self.cmd_details))
        # 为退出件添加事件
        self.views.but_cancle.add_event(click_EventHandler(self.views.but_cancle, self.exit))

    def cmd_details(self, view, event):
        global cmd_position
        # 获取用户点击的菜单项位置
        cmd_position = event['data']['position']
        # 关闭当前界面
        FullScreenWrapper2App.close_layout()

    def on_close(self):
        pass

    def exit(self, view, dummy):
        droid = FullScreenWrapper2App.get_android_instance()
        droid.makeToast("Exit")
        # 关闭界面
        FullScreenWrapper2App.close_layout()
        global cmd_position
        # 设置全局变量
        cmd_position = -1


class CommandDetails(Layout):
    def __init__(self):
        super(CommandDetails, self).__init__(detail_content, "DetailsApp")

    def on_show(self):
        list_view = self.views.listview
        droid = FullScreenWrapper2App.get_android_instance()
        position = int(cmd_position)
        title = 'Details'
        try:
            # 根据全局变量cmd_position获取对应的命令
            title = INDEX_PAGE_LIST[position]
            cmd = title.lower().replace(' ', '_')
            cmd_res = GetCommandResult(cmd, droid)
            # 获取命令对应的执行结果
            show_list_items = cmd_res.get_result()
            # 获取命令对应的文档
            code = cmd_res.get_doc()
            # 将代码设置到界面的代码显示框中
            self.views.codeTextView.text = code
        except Exception as e:
            show_list_items = [f'ServerError: {str(e)}']
        # 将命令执行结果设置到页面
        list_view.set_listitems(show_list_items)
        # 将命令设置为页面标题
        self.views.Title.text = title
        # 为复制按键添加事件
        self.views.but_copy.add_event(click_EventHandler(self.views.but_copy, self.copy_code))
        # 为返回按键添加事件
        self.views.but_cancle.add_event(click_EventHandler(self.views.but_cancle, self.back))
        # 为执行按键添加事件
        self.views.but_try.add_event(click_EventHandler(self.views.but_try, self.retry))

    def on_close(self):
        pass

    def back(self, view, dummy):
        # 关闭当前页面
        FullScreenWrapper2App.close_layout()

    def copy_code(self, view, dummy):
        droid = FullScreenWrapper2App.get_android_instance()
        # 将代码框中的文本设置到剪贴板
        droid.setClipboard(self.views.codeTextView.text)
        droid.makeToast("Copied")

    def retry(self, view, dummy):
        # 执行命令，显示结果
        list_view = self.views.listview
        droid = FullScreenWrapper2App.get_android_instance()
        position = int(cmd_position)
        try:
            title = INDEX_PAGE_LIST[position]
            cmd = title.lower().replace(' ', '_')
            cmd_res = GetCommandResult(cmd, droid)
            show_list_items = cmd_res.get_result()
            code = cmd_res.get_doc()
            self.views.codeTextView.text = code
        except Exception as e:
            show_list_items = [f'ServerError: {str(e)}']
        list_view.set_listitems(show_list_items)


def index_page():
    # 运行主界面
    droid = androidhelper.Android()
    FullScreenWrapper2App.initialize(droid)
    FullScreenWrapper2App.show_layout(MainScreen())
    FullScreenWrapper2App.eventloop()
    return cmd_position


def detail_page():
    # 运行命令详情页面
    droid = androidhelper.Android()
    FullScreenWrapper2App.initialize(droid)
    FullScreenWrapper2App.show_layout(CommandDetails())
    FullScreenWrapper2App.eventloop()


if __name__ == '__main__':
    while True:
        position = index_page()
        if position == -1:  # 退出应用
            break
        detail_page()
